<a href="https://colab.research.google.com/drive/1PBvBa2Gt9NtNIZGnfr-7uYui_fRSCyJI?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

#	🔢 What is Multi-index?

Multi-index is a RAG technique that involves using multiple separate indexes or databases to store and retrieve information. Instead of having a single, monolithic index, data is distributed across several specialized indexes. Each index can be optimized for different types of data or query patterns, allowing for more efficient and targeted.

# 🔧 Multi-index RAG Implementation:

1. **Retrieval from Multiple Indices:** We retrieve relevant documents from each vector store.
2. **Context Combination:** We combine the retrieved documents from all sources into a single context.
3. **Response Generation:** Using the combined context, we generate a final response to the original query.
4. **Source Usage Explanation:** We generate an explanation of how information from different sources was used to answer the question.

# ⚙️ Setup

1. **[LLM](https://deepmind.google/technologies/gemini/pro/):** Google's free gemini-pro api endpoint ([Google's API Key](https://console.cloud.google.com/apis/credentials))
2. **[Vector Store](https://www.pinecone.io/learn/vector-database/):** [ChromaDB](https://www.trychroma.com/)
3. **[Embedding Model](https://qdrant.tech/articles/what-are-embeddings/):** [nomic-embed-text-v1.5](https://www.nomic.ai/blog/posts/nomic-embed-text-v1)
4. **[LLM Framework](https://python.langchain.com/v0.2/docs/introduction/):** LangChain
5. **[Huggingface API Key](https://huggingface.co/settings/tokens)**

# Install required libraries

In [ ]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.2.11 \
     langchain-google-genai==1.0.7 \
     langchain-chroma==0.1.2 \
     langchain-community==0.2.10 \
     langchain-huggingface==0.0.3 \
     einops==0.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 13.3 MB/s et

# Import related libraries related to Langchain, HuggingfaceEmbedding

In [ ]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import WebBaseLoader

In [ ]:
import os
import getpass

# Provide Google API Key. You can create Google API key at following link

[Google Gemini-Pro API Creation Link](https://console.cloud.google.com/apis/credentials)

[YouTube Video](https://www.youtube.com/watch?v=ZHX7zxvDfoc)



In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


# Provide Huggingface API Key. You can create Huggingface API key at following link

[Huggingface API Creation Link](https://huggingface.co/settings/tokens)




In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


# Step 1: Load and preprocess data code

In [ ]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    return chunks

# Step 2: Create multiple vector stores code

In [ ]:
def create_vector_stores(chunks_list):
    embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
    vector_stores = []
    for chunks in chunks_list:
        vector_store = Chroma.from_documents(chunks, embeddings)
        vector_stores.append(vector_store)
    return vector_stores

# Step 3: Mult-index-RAG related code

1. **Retrieval from Multiple Indices:** We retrieve relevant documents from each vector store.
2. **Context Combination:** We combine the retrieved documents from all sources into a single context.
3. **Response Generation:** Using the combined context, we generate a final response to the original query.
4. **Source Usage Explanation:** We generate an explanation of how information from different sources was used to answer the question.

In [ ]:
def multi_index_rag(query, vector_stores, llm):
    # Retrieve documents from each vector store
    all_docs = []
    for i, vector_store in enumerate(vector_stores):
        docs = vector_store.similarity_search(query, k=3)  # Increased k to 3
        all_docs.extend([f"Source {i+1}: " + doc.page_content for doc in docs])

    # Combine retrieved documents
    context = "\n\n".join(all_docs)

    # Generate response using combined context
    response_prompt = ChatPromptTemplate.from_template(
        "You are an AI assistant tasked with answering questions based on the provided context. "
        "The context contains information from multiple sources related to AI, machine learning, and NLP. "
        "Please analyze the context carefully and provide a comprehensive answer to the question. "
        "If the context doesn't contain enough information, use your general knowledge to supplement the answer, "
        "but prioritize information from the context when available.\n\n"
        "Context:\n{context}\n\n"
        "Question: {query}\n"
        "Answer:"
    )
    response_chain = response_prompt | llm
    try:
        response = response_chain.invoke({"context": context, "query": query})
        final_answer = response.content
    except Exception as e:
        print(f"Error generating response: {e}")
        final_answer = "I apologize, but I encountered an error while generating the response."

    # Generate explanation of source usage
    explanation_prompt = ChatPromptTemplate.from_template(
        "Based on the answer you provided, explain how information from different sources was used to answer the question. "
        "If general knowledge was used to supplement the answer, mention that as well.\n\n"
        "Context: {context}\n"
        "Question: {query}\n"
        "Answer: {answer}\n"
        "Explanation:"
    )
    explanation_chain = explanation_prompt | llm
    try:
        explanation = explanation_chain.invoke({"context": context, "query": query, "answer": final_answer})
        source_explanation = explanation.content
    except Exception as e:
        print(f"Error generating explanation: {e}")
        source_explanation = "Unable to generate explanation due to an error."

    return {
        "final_answer": final_answer,
        "source_explanation": source_explanation,
        "retrieved_context": context
    }

# Step 4: Create chunk of web data to Chroma Vector Store

In [ ]:
# Initialize the gemini-pro language model with specified settings (Change temeprature  and other parameters as per your requirement)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, safety_settings={
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        },)

# Load and process data
urls = [
        "https://en.wikipedia.org/wiki/Artificial_intelligence",
        "https://en.wikipedia.org/wiki/Machine_learning",
        "https://en.wikipedia.org/wiki/Natural_language_processing"
      ]
chunks_list = [load_and_process_data(url) for url in urls]

# Create multiple vector stores
vector_stores = create_vector_stores(chunks_list)

# Step 5: Run Multi-index RAG

This implementation shows the key parts of Multi-index RAG:

1. Use of multiple vector stores representing different sources or types of information
2. Retrieval and combination of information from multiple sources
3. Generation of a comprehensive response using the combined information
Explanation of how different sources contributed to the answer

In [ ]:
# Example query
query = "How do AI, machine learning, and NLP relate to each other in modern applications?"

# Run Multi-index RAG
result = multi_index_rag(query, vector_stores, llm)

print("Query:", query)
print("\nFinal Answer:")
print(result["final_answer"])
print("\nSource Usage Explanation:")
print(result["source_explanation"])

Query: How do AI, machine learning, and NLP relate to each other in modern applications?

Final Answer:
Machine learning is a subfield of AI that allows programs to improve their performance on a given task automatically. NLP, in turn, is a subfield of AI and computer science that focuses on providing computers the ability to process data encoded in natural language. In modern applications, AI, machine learning, and NLP are often used together to create systems that can understand and respond to human language. For example, AI-powered chatbots use machine learning to improve their ability to understand and respond to user queries, and NLP to process and interpret the user's input.

Source Usage Explanation:
The answer draws upon information from all three sources to provide a comprehensive explanation of the relationship between AI, machine learning, and NLP in modern applications.

- Source 1 defines machine learning as a subfield of AI and NLP as a subfield of AI and computer science